## CNN (1D)

In [38]:
!pip install keras


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Charger les données
train_texts = df_pandas['text'].values
train_labels = df_pandas['sentiment'].values

# Paramètres de tokenization
max_words = 20000  # Nombre maximum de mots dans le vocabulaire
max_len = 100      # Longueur maximale des séquences

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_texts)
sequences = tokenizer.texts_to_sequences(train_texts)

# Padding des séquences
X = pad_sequences(sequences, maxlen=max_len)

# Encodage des labels en format one-hot
encoder = OneHotEncoder(sparse_output=False)

y = encoder.fit_transform(train_labels.reshape(-1, 1))

# Diviser en ensembles de formation, validation et test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Convertir les données en tenseurs PyTorch
X_train = torch.tensor(X_train, dtype=torch.long)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.long)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Créer un Dataset PyTorch
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = TextDataset(X_train, y_train)
val_dataset = TextDataset(X_val, y_val)
test_dataset = TextDataset(X_test, y_test)

# Dataloader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# Construire le modèle PyTorch
class CNN1D(nn.Module):
    def __init__(self, vocab_size, embed_size, num_classes):
        super(CNN1D, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.conv1d = nn.Conv1d(in_channels=embed_size, out_channels=64, kernel_size=5)
        self.global_max_pool = nn.AdaptiveMaxPool1d(1)
        self.fc1 = nn.Linear(64, 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.embedding(x).permute(0, 2, 1)  # Convertir en [batch_size, embed_size, seq_len]
        x = torch.relu(self.conv1d(x))
        x = self.global_max_pool(x).squeeze(-1)  # [batch_size, out_channels]
        x = self.dropout(torch.relu(self.fc1(x)))
        x = self.fc2(x)
        return torch.softmax(x, dim=1)

# Initialiser le modèle
model = CNN1D(vocab_size=max_words, embed_size=128, num_classes=3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entraîner le modèle
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=50):
    for epoch in range(epochs):
        model.train()
        train_loss, train_acc = 0, 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, torch.argmax(y_batch, dim=1))
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_acc += (outputs.argmax(dim=1) == y_batch.argmax(dim=1)).float().mean().item()

        val_loss, val_acc = 0, 0
        model.eval()
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, torch.argmax(y_batch, dim=1))

                val_loss += loss.item()
                val_acc += (outputs.argmax(dim=1) == y_batch.argmax(dim=1)).float().mean().item()

        print(f"Epoch {epoch+1}/{epochs}, "
              f"Train Loss: {train_loss/len(train_loader):.4f}, "
              f"Train Acc: {train_acc/len(train_loader):.4f}, "
              f"Val Loss: {val_loss/len(val_loader):.4f}, "
              f"Val Acc: {val_acc/len(val_loader):.4f}")

# Appeler la fonction d'entraînement
train_model(model, train_loader, val_loader, criterion, optimizer)


Epoch 1/50, Train Loss: 1.0098, Train Acc: 0.5005, Val Loss: 0.9275, Val Acc: 0.5998
Epoch 2/50, Train Loss: 0.8898, Train Acc: 0.6504, Val Loss: 0.8921, Val Acc: 0.6439
Epoch 3/50, Train Loss: 0.8182, Train Acc: 0.7262, Val Loss: 0.8810, Val Acc: 0.6527
Epoch 4/50, Train Loss: 0.7703, Train Acc: 0.7792, Val Loss: 0.8813, Val Acc: 0.6568
Epoch 5/50, Train Loss: 0.7330, Train Acc: 0.8169, Val Loss: 0.8840, Val Acc: 0.6571
Epoch 6/50, Train Loss: 0.7103, Train Acc: 0.8418, Val Loss: 0.8809, Val Acc: 0.6574
Epoch 7/50, Train Loss: 0.6942, Train Acc: 0.8569, Val Loss: 0.8746, Val Acc: 0.6673
Epoch 8/50, Train Loss: 0.6815, Train Acc: 0.8699, Val Loss: 0.8806, Val Acc: 0.6616
Epoch 9/50, Train Loss: 0.6726, Train Acc: 0.8775, Val Loss: 0.8839, Val Acc: 0.6565
Epoch 10/50, Train Loss: 0.6681, Train Acc: 0.8833, Val Loss: 0.8778, Val Acc: 0.6673
Epoch 11/50, Train Loss: 0.6608, Train Acc: 0.8906, Val Loss: 0.8800, Val Acc: 0.6629
Epoch 12/50, Train Loss: 0.6574, Train Acc: 0.8929, Val Loss: 0

In [41]:
from sklearn.metrics import classification_report, accuracy_score

def evaluate_model(model, data_loader, criterion):
    model.eval()  # Met le modèle en mode évaluation
    total_loss = 0
    all_targets = []
    all_predictions = []

    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            outputs = model(X_batch)
            loss = criterion(outputs, torch.argmax(y_batch, dim=1))
            total_loss += loss.item()

            # Stocker les vraies classes et les prédictions
            all_targets.extend(torch.argmax(y_batch, dim=1).cpu().numpy())
            all_predictions.extend(torch.argmax(outputs, dim=1).cpu().numpy())

    # Calculer la perte moyenne et la précision
    avg_loss = total_loss / len(data_loader)
    accuracy = accuracy_score(all_targets, all_predictions)

    # Afficher le rapport détaillé
    print("Classification Report:")
    print(classification_report(all_targets, all_predictions))

    return avg_loss, accuracy


In [42]:
# Évaluer sur l'ensemble d'entraînement
train_loss, train_accuracy = evaluate_model(model, train_loader, criterion)
print(f"Performance sur l'ensemble d'entraînement : Perte = {train_loss:.4f}, Précision = {train_accuracy:.4f}")

# Évaluer sur l'ensemble de validation
val_loss, val_accuracy = evaluate_model(model, val_loader, criterion)
print(f"Performance sur l'ensemble de validation : Perte = {val_loss:.4f}, Précision = {val_accuracy:.4f}")

# Évaluer sur l'ensemble de test
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f"Performance sur l'ensemble de test : Perte = {test_loss:.4f}, Précision = {test_accuracy:.4f}")


Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.93      0.93      6170
           1       0.94      0.93      0.93      6131
           2       0.94      0.93      0.94      6169

    accuracy                           0.93     18470
   macro avg       0.93      0.93      0.93     18470
weighted avg       0.93      0.93      0.93     18470

Performance sur l'ensemble d'entraînement : Perte = 0.6192, Précision = 0.9319
Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.60      0.58      1308
           1       0.74      0.77      0.75      1314
           2       0.71      0.63      0.67      1336

    accuracy                           0.67      3958
   macro avg       0.67      0.67      0.67      3958
weighted avg       0.67      0.67      0.67      3958

Performance sur l'ensemble de validation : Perte = 0.8844, Précision = 0.6655
Classification Report:
            

In [44]:
import torch
import numpy as np

# Préparation des séquences et du padding
new_texts = ["I love this!", "This is terrible!", "It’s okay."]
new_sequences = tokenizer.texts_to_sequences(new_texts)
new_padded = pad_sequences(new_sequences, maxlen=max_len)

# Conversion en tenseur PyTorch
new_padded_tensor = torch.tensor(new_padded, dtype=torch.long)  

# Mode évaluation
model.eval()

# Prédictions
with torch.no_grad():  # Désactive le calcul des gradients pour accélérer les prédictions
    predictions = model(new_padded_tensor)

# Affichage des résultats
for text, pred in zip(new_texts, predictions):
    predicted_class = torch.argmax(pred).item()  # Trouver la classe prédite
    print(f"Text: {text}")
    print(f"Predicted Sentiment: {predicted_class}")



Text: I love this!
Predicted Sentiment: 1
Text: This is terrible!
Predicted Sentiment: 2
Text: It’s okay.
Predicted Sentiment: 0


## CNN 2D

In [49]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Charger les données
train_texts = df_pandas['text'].values
train_labels = df_pandas['sentiment'].values

# Paramètres de tokenization
max_words = 20000  # Nombre maximum de mots dans le vocabulaire
max_len = 100      # Longueur maximale des séquences

# Tokenization
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_texts)
sequences = tokenizer.texts_to_sequences(train_texts)

# Padding des séquences
X = pad_sequences(sequences, maxlen=max_len)

# Encodage des labels en format one-hot
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(train_labels.reshape(-1, 1))

# Conversion en tenseurs PyTorch
X = torch.tensor(X, dtype=torch.float32).unsqueeze(1)  # Ajout du canal unique
X = X.permute(0, 1, 2).unsqueeze(1)  # Réorganise en [batch_size, 1, height, width]
y = torch.tensor(y, dtype=torch.float32)

# Diviser en ensembles de formation, validation et test
dataset = torch.utils.data.TensorDataset(X, y)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

class CNN2D(nn.Module):
    def __init__(self, num_classes=3):
        super(CNN2D, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=(1, 1))  # Ajustement du noyau à (1, 1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=(1, 1))  # Ajustement du noyau à (1, 1)
        self.global_pool = nn.AdaptiveMaxPool2d((1, 1))
        self.fc1 = nn.Linear(64, 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.conv1(x))  # Appliquer ReLU après conv1
        x = self.dropout(x)
        x = self.relu(self.conv2(x))  # Appliquer ReLU après conv2
        x = self.global_pool(x)
        x = x.view(x.size(0), -1)  # Aplatir les tenseurs pour les couches entièrement connectées
        x = self.relu(self.fc1(x))  # ReLU après fc1
        x = self.dropout(x)
        x = self.fc2(x)  # Pas de ReLU ici, car softmax suit
        return self.softmax(x)



# Initialiser le modèle
model = CNN2D(num_classes=3)

# Définir l'optimiseur et la fonction de perte
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Entraîner le modèle
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels.argmax(dim=1))
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels.argmax(dim=1)).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    accuracy = correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}")


Epoch 1/20, Loss: 1.2166, Accuracy: 0.3349
Epoch 2/20, Loss: 1.2173, Accuracy: 0.3342
Epoch 3/20, Loss: 1.2175, Accuracy: 0.3339
Epoch 4/20, Loss: 1.2174, Accuracy: 0.3341
Epoch 5/20, Loss: 1.2175, Accuracy: 0.3340
Epoch 6/20, Loss: 1.2175, Accuracy: 0.3339
Epoch 7/20, Loss: 1.2175, Accuracy: 0.3339
Epoch 8/20, Loss: 1.2175, Accuracy: 0.3339
Epoch 9/20, Loss: 1.2175, Accuracy: 0.3339
Epoch 10/20, Loss: 1.2164, Accuracy: 0.3349
Epoch 11/20, Loss: 1.2178, Accuracy: 0.3336
Epoch 12/20, Loss: 1.2175, Accuracy: 0.3339
Epoch 13/20, Loss: 1.2174, Accuracy: 0.3341
Epoch 14/20, Loss: 1.2175, Accuracy: 0.3339
Epoch 15/20, Loss: 1.2174, Accuracy: 0.3340
Epoch 16/20, Loss: 1.2177, Accuracy: 0.3337
Epoch 17/20, Loss: 1.2174, Accuracy: 0.3340
Epoch 18/20, Loss: 1.2173, Accuracy: 0.3341
Epoch 19/20, Loss: 1.2173, Accuracy: 0.3342
Epoch 20/20, Loss: 1.2177, Accuracy: 0.3337


In [50]:
# Évaluation de la performance sur l'ensemble d'entraînement
model.eval()  # Passer le modèle en mode évaluation
train_loss = 0.0
train_correct = 0
train_total = 0

with torch.no_grad():
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels.argmax(dim=1))

        train_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        train_correct += predicted.eq(labels.argmax(dim=1)).sum().item()
        train_total += labels.size(0)

train_loss /= len(train_loader.dataset)
train_accuracy = train_correct / train_total
print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

# Évaluation de la performance sur l'ensemble de test
test_loss = 0.0
test_correct = 0
test_total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels.argmax(dim=1))

        test_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        test_correct += predicted.eq(labels.argmax(dim=1)).sum().item()
        test_total += labels.size(0)

test_loss /= len(test_loader.dataset)
test_accuracy = test_correct / test_total
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Train Loss: 1.2175, Train Accuracy: 0.3339
Test Loss: 1.2206, Test Accuracy: 0.3309


In [53]:
import torch
import numpy as np

# Préparation des séquences et du padding
new_texts = ["I love this!", "This is terrible!", "It’s okay."]
new_sequences = tokenizer.texts_to_sequences(new_texts)
new_padded = pad_sequences(new_sequences, maxlen=max_len)

# Conversion en tenseur PyTorch (conversion en float32)
new_padded_tensor = torch.tensor(new_padded, dtype=torch.float32)  # Convertir en float32

# Ajouter une dimension de canal (formater comme (batch_size, 1, height, width))
new_padded_tensor = new_padded_tensor.unsqueeze(1)  # Ajouter la dimension du canal (1, height, width)

# La forme attendue par la couche Conv2d est (batch_size, 1, max_len, 1)
new_padded_tensor = new_padded_tensor.unsqueeze(3)  # Ajout de la profondeur pour correspondre à (batch_size, 1, max_len, 1)

# Mode évaluation
model.eval()

# Prédictions
with torch.no_grad():  # Désactive le calcul des gradients pour accélérer les prédictions
    predictions = model(new_padded_tensor)

# Affichage des résultats
for text, pred in zip(new_texts, predictions):
    predicted_class = torch.argmax(pred).item()  # Trouver la classe prédite
    print(f"Text: {text}")
    print(f"Predicted Sentiment: {predicted_class}")


Text: I love this!
Predicted Sentiment: 1
Text: This is terrible!
Predicted Sentiment: 1
Text: It’s okay.
Predicted Sentiment: 1


## CNN 3D

In [57]:
!pip install torchtext


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.9 MB/s eta 0:00:00


In [76]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Charger les données
train_texts = df_pandas['text'].values
train_labels = df_pandas['sentiment'].values

# Paramètres de tokenization
max_words = 20000  # Nombre maximum de mots dans le vocabulaire
max_len = 100      # Longueur maximale des séquences
embedding_dim = 128  # Taille des vecteurs d'embedding

# Tokenization
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_texts)
sequences = tokenizer.texts_to_sequences(train_texts)

# Padding des séquences
X = pad_sequences(sequences, maxlen=max_len)

# Encoder les labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train_labels)

# Convertir en one-hot
y = np.eye(3)[y]

# Ajouter une dimension pour les embeddings (simuler 3D)
X = np.expand_dims(X, axis=-1)  # (num_samples, max_len, 1)
X = np.expand_dims(X, axis=-1)  # (num_samples, max_len, 1, 1)

# Convertir les données en tensors PyTorch
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)


X_tensor = X_tensor.unsqueeze(1)  # Shape devient (batch_size, 1, max_len, 1, 1)

# Diviser les données en ensembles d'entraînement, validation et test
X_train, X_temp, y_train, y_temp = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Créer des DataLoader pour les lots
train_data = TensorDataset(X_train, y_train)
val_data = TensorDataset(X_val, y_val)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Définir le modèle CNN 3D
class CNN3DModel(nn.Module):
    def __init__(self):
        super(CNN3DModel, self).__init__()

        # Définir les couches du modèle
        self.conv1 = nn.Conv3d(1, 32, kernel_size=(3, 1, 1))  
        self.pool1 = nn.MaxPool3d(kernel_size=(2, 1, 1))
        self.conv2 = nn.Conv3d(32, 64, kernel_size=(3, 1, 1)) 
        self.pool2 = nn.MaxPool3d(kernel_size=(2, 1, 1))

        # Calculate the output size dynamically by passing a dummy tensor
        self.dummy_input = torch.zeros(1, 1, 100, 1, 1)  # Dummy input tensor
        self.conv_output_size = self._get_conv_output(self.dummy_input)

        # Fully connected layers
        self.fc1 = nn.Linear(self.conv_output_size, 128)  # Updated to match the output of the convolutions
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 3)  # Sortie pour 3 classes

    def _get_conv_output(self, shape):
        # Pass the dummy input through the conv layers to calculate the output size
        x = self.conv1(shape)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        return int(np.prod(x.size()))  # Flatten the tensor and return the number of features

    def forward(self, x):
        # Passer par les couches
        x = self.pool1(torch.relu(self.conv1(x)))  # Appliquer ReLU après Conv3d
        x = self.pool2(torch.relu(self.conv2(x)))  # Appliquer ReLU après Conv3d
        x = torch.flatten(x, 1)  # Aplatissement avant les couches entièrement connectées
        x = torch.relu(self.fc1(x))  # Appliquer ReLU après la couche FC
        x = self.dropout1(x)
        x = self.fc2(x)  # Pas d'activation ici, car on l'ajoutera dans la fonction de perte
        return x

# Initialiser le modèle
model = CNN3DModel()

# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entraînement du modèle avec évaluation sur l'ensemble de test
def train(model, train_loader, val_loader, test_loader, epochs=20):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.argmax(dim=1))
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels.argmax(dim=1)).sum().item()

        train_accuracy = correct / total
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.4f}")

        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels.argmax(dim=1)).sum().item()

        val_accuracy = correct / total
        print(f"Validation Accuracy: {val_accuracy:.4f}")

        # Test Evaluation (after every epoch)
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels.argmax(dim=1)).sum().item()

        test_accuracy = correct / total
        print(f"Test Accuracy: {test_accuracy:.4f}")

# Entraîner le modèle avec test évaluation
train(model, train_loader, val_loader, test_loader)


Epoch [1/20], Loss: 2.6014, Accuracy: 0.3621
Validation Accuracy: 0.4036
Test Accuracy: 0.4051
Epoch [2/20], Loss: 1.0723, Accuracy: 0.3979
Validation Accuracy: 0.4236
Test Accuracy: 0.4377
Epoch [3/20], Loss: 1.0578, Accuracy: 0.4044
Validation Accuracy: 0.4187
Test Accuracy: 0.4388
Epoch [4/20], Loss: 1.0544, Accuracy: 0.4099
Validation Accuracy: 0.4221
Test Accuracy: 0.4331
Epoch [5/20], Loss: 1.0493, Accuracy: 0.4116
Validation Accuracy: 0.3759
Test Accuracy: 0.3903
Epoch [6/20], Loss: 1.0472, Accuracy: 0.4116
Validation Accuracy: 0.4293
Test Accuracy: 0.4487
Epoch [7/20], Loss: 1.0511, Accuracy: 0.4089
Validation Accuracy: 0.3312
Test Accuracy: 0.3452
Epoch [8/20], Loss: 1.0933, Accuracy: 0.3482
Validation Accuracy: 0.3850
Test Accuracy: 0.3945
Epoch [9/20], Loss: 1.0775, Accuracy: 0.3676
Validation Accuracy: 0.3331
Test Accuracy: 0.3471
Epoch [10/20], Loss: 1.0935, Accuracy: 0.3404
Validation Accuracy: 0.4202
Test Accuracy: 0.4399
Epoch [11/20], Loss: 1.0636, Accuracy: 0.3900
Val

# CNN+LSTM

In [77]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import numpy as np
from torch.nn import functional as F

# Paramètres
max_words = 20000  # Taille du vocabulaire
max_len = 100      # Longueur maximale des séquences
embedding_dim = 128
num_classes = 3    # Nombre de classes (par exemple, sentiment)

# Charger les données
train_texts = df_pandas['text'].values
train_labels = df_pandas['sentiment'].values

# Tokenization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_texts)
sequences = tokenizer.texts_to_sequences(train_texts)
X = pad_sequences(sequences, maxlen=max_len)

# Encodage des labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train_labels)
y = np.eye(num_classes)[y]  # One-hot encoding

# Convertir en tensors PyTorch
X_tensor = torch.tensor(X, dtype=torch.long)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Diviser les données en ensembles de formation, validation et test
X_train, X_temp, y_train, y_temp = train_test_split(X_tensor, y_tensor, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Créer des DataLoader pour les lots
train_data = TensorDataset(X_train, y_train)
val_data = TensorDataset(X_val, y_val)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Définir le modèle CNN + LSTM avec PyTorch
class CNNLSTMModel(nn.Module):
    def __init__(self):
        super(CNNLSTMModel, self).__init__()
        self.embedding = nn.Embedding(max_words, embedding_dim)

        # Partie CNN
        self.conv1 = nn.Conv1d(in_channels=embedding_dim, out_channels=64, kernel_size=5)
        self.pool = nn.MaxPool1d(4)

        # Partie LSTM
        self.lstm = nn.LSTM(input_size=64, hidden_size=100, batch_first=True)

        # Partie Dense pour la classification
        self.fc1 = nn.Linear(100, 64)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(0, 2, 1)  # Changer les dimensions pour Conv1d

        # Partie CNN
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = x.permute(0, 2, 1)  # Revenir aux dimensions pour LSTM

        # Partie LSTM
        x, (h_n, c_n) = self.lstm(x)
        x = h_n[-1]  # Utiliser la dernière sortie du LSTM

        # Partie Dense
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # Pas de ReLU ici, car nous appliquons softmax dans la fonction de perte
        return x

# Initialiser le modèle
model = CNNLSTMModel()

# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()  # Utilisation de CrossEntropyLoss pour la classification multi-classes
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entraînement du modèle
def train(model, train_loader, val_loader, epochs=20):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.argmax(dim=1))  # Utilisation de .argmax() pour obtenir l'étiquette de classe
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels.argmax(dim=1)).sum().item()

        train_accuracy = correct / total
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.4f}")

        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels.argmax(dim=1)).sum().item()

        val_accuracy = correct / total
        print(f"Validation Accuracy: {val_accuracy:.4f}")

# Entraîner le modèle
train(model, train_loader, val_loader)

# Évaluation sur les données de test
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels.argmax(dim=1)).sum().item()

    test_accuracy = correct / total
    print(f"Test Accuracy: {test_accuracy:.4f}")

# Évaluation sur l'ensemble de test
evaluate(model, test_loader)


Epoch [1/20], Loss: 0.9338, Accuracy: 0.5350
Validation Accuracy: 0.6071
Epoch [2/20], Loss: 0.6919, Accuracy: 0.6977
Validation Accuracy: 0.6592
Epoch [3/20], Loss: 0.5173, Accuracy: 0.7895
Validation Accuracy: 0.6609
Epoch [4/20], Loss: 0.3610, Accuracy: 0.8631
Validation Accuracy: 0.6493
Epoch [5/20], Loss: 0.2394, Accuracy: 0.9134
Validation Accuracy: 0.6508
Epoch [6/20], Loss: 0.1623, Accuracy: 0.9434
Validation Accuracy: 0.6470
Epoch [7/20], Loss: 0.1214, Accuracy: 0.9567
Validation Accuracy: 0.6458
Epoch [8/20], Loss: 0.0895, Accuracy: 0.9680
Validation Accuracy: 0.6534
Epoch [9/20], Loss: 0.0628, Accuracy: 0.9795
Validation Accuracy: 0.6440
Epoch [10/20], Loss: 0.0603, Accuracy: 0.9793
Validation Accuracy: 0.6536
Epoch [11/20], Loss: 0.0574, Accuracy: 0.9798
Validation Accuracy: 0.6521
Epoch [12/20], Loss: 0.0445, Accuracy: 0.9849
Validation Accuracy: 0.6453
Epoch [13/20], Loss: 0.0455, Accuracy: 0.9843
Validation Accuracy: 0.6488
Epoch [14/20], Loss: 0.0369, Accuracy: 0.9873
V